PATH ridership information is published as a PDF which makes it so unbelievably inconvenient to work with for analytic purposes.
PATH ridership data: https://www.panynj.gov/path/statistics.html

In [189]:
# tabula requires java
import tabula
import pandas as pd
import numpy as np

In [185]:
#inputs
pdf_path = "../data/path_ridership_data/2019-PATH-hourly-Ridership-Report.pdf"
file_name_output = '2019-PATH-hourly-Ridership-Report.csv'

#constants
months = [1,2,3,4,5,6,7,8,9,10,11,12]

In [181]:
tables = tabula.read_pdf(
    pdf_path,
    multiple_tables=True,
    encoding='ISO-8859-1',
    pages='all')

print(len(tables))

126


In [186]:
def getColumnNames(first_row, second_row):
    length = min(len(first_row), len(second_row))
    columnNames = []
    for index in np.arange(length):
        words1 = first_row[index]
        words2 = second_row[index]
        if pd.isnull(words1):
            words1 = ""
        if pd.isnull(words2):
            words2 = ""
        if words1 == "" and words2 == "":
            columnNames.append("BLANK")
        elif words1 == "" or words2 == "":
            columnNames.append(words1 + words2)
        else:
            columnNames.append(words1 + " " + words2)
    return columnNames

station_data = {}

for table in tables:
    station_name = table.iloc[0][1]
    data = []
    if station_name in station_data:
        data = station_data[station_name]
    
    names = getColumnNames(table.iloc[1], table.iloc[2])
    while True:
        if "BLANK" in names:
            blank_index = names.index("BLANK")
            table = table.drop([table.columns[blank_index]] ,  axis='columns')
            names.pop(blank_index)
        else:
            break
    table.columns = names
  
    table = table.iloc[3:len(table)-1,:]
    table = table.reset_index(drop=True)
    table = table.fillna(0)
    data.append(table)
    station_data[station_name] = data


In [187]:
# now we need to add month, year, and station name to the 
# dataframes so we can later combine them into one csv file
# Note: if you want to rerun this cell, rerun the cell above first

# idk why Tabula splits System-wide into two tables    
station_data.pop('Avg Weekday', None)
station_data.pop('System?wide', None)

# for station_name in station_data:
#     #spot check, all stations should have 13 stations
#     print(station_name, len(station_data[station_name]))

for station_name in station_data:
    data = station_data[station_name]
    # first value are the year-end stats
    data.pop(0)
    for index in np.arange(len(data)):
        data[index]['month'] = months[index]
        data[index]['station'] = station_name    

In [184]:
data_per_station = []
for station_name in station_data:
    data = station_data[station_name]
    data_per_station.append(pd.concat(data))
        
        
master_df = pd.concat(data_per_station)
len(master_df)

master_df.to_csv(file_name_output, index=False)
